Импорт библиотек

In [1]:
!pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 76.6 MB 114 kB/s 


In [1]:
import pandas as pd
from sklearn.pipeline import Pipeline
import warnings
from catboost import CatBoostRegressor, cv, Pool, CatBoostClassifier
import numpy as np
import sys
from sklearn.preprocessing import FunctionTransformer
from sklearn.model_selection import train_test_split

Считывание необходимых файлов

In [2]:
df = pd.read_csv('train_dataset_train.csv',sep=',')
df.head()

,id,ticket_id,ticket_type_nm,entrance_id,entrance_nm,station_id,station_nm,line_id,line_nm,pass_dttm,time_to_under,label
0,1,40BD89EC85646EFB69E283F39C298E60,Пропуск FacePay,2402,Лефортово БКЛ,11007,Лефортово,11,Большая кольцевая,2022-09-12 05:00:13,216.316667,8001
1,2,126727A96489CC976A8C08E5CEB00542,СК учащегося 30 дней,110,Войковская ( Южный ),2006,Войковская,2,Замоскворецкая,2022-09-12 05:00:54,648.183333,9011
2,3,D28CE6A9E0E5B6D213470A97CFF32485,БСК дружинника г.Москвы,110,Войковская ( Южный ),2006,Войковская,2,Замоскворецкая,2022-09-12 05:00:55,865.333333,7022
3,4,015DA44B523C062B5BFEFF3FB0E64B9E,30 дней,110,Войковская ( Южный ),2006,Войковская,2,Замоскворецкая,2022-09-12 05:01:13,1048.233333,2022
4,5,95B19C6F3A504727AC3EA56EB7E3E80F,КОШЕЛЕК,110,Войковская ( Южный ),2006,Войковская,2,Замоскворецкая,2022-09-12 05:02:55,965.600000,2017


In [3]:
print(df.shape[0])

1091021


Посмотрим общее описание

In [4]:
df.describe(include='O')

,ticket_id,ticket_type_nm,entrance_nm,station_nm,line_nm,pass_dttm
count,1091021,1091021,1091021,1091021,1091021,1091021
unique,335533,60,426,245,16,297040
top,7992E92F9AE0F7506BD439547FD7E11F,КОШЕЛЕК,Щёлковская ( Северный ),Щёлковская,Таганско-Краснопресненская,2022-09-12 08:20:09
freq,8,262499,17810,22133,156031,29


id билета разный для всех пользователей. Следовательно, он не применим не только для задачи регрессии, но и для задачи классификации. Следовательно, в дальнейшем удалим его.

Удалим дубликаты по id билета и дате

In [5]:
df.drop_duplicates(subset=['ticket_id', 'pass_dttm'],inplace=True)
print(df.shape[0])

1091021


Посмотрим, сколько nan значений

In [6]:
df.isnull().sum()

id                0
ticket_id         0
ticket_type_nm    0
entrance_id       0
entrance_nm       0
station_id        0
station_nm        0
line_id           0
line_nm           0
pass_dttm         0
time_to_under     0
label             0
dtype: int64

Удалим nan значения

In [7]:
df.dropna(inplace=True)

Избавляемся от дублирующих по смыслу колонок, а именно id станции, входа в станцию и ветки. Также id билета, тк это бесполезный параметр(тк id для каждого билета свой)

In [8]:
df.drop(columns=['ticket_id', 'entrance_id', 'station_id','line_id'],inplace=True)
df.pass_dttm = pd.to_datetime(df.pass_dttm)

In [9]:
df.head()

,id,ticket_type_nm,entrance_nm,station_nm,line_nm,pass_dttm,time_to_under,label
0,1,Пропуск FacePay,Лефортово БКЛ,Лефортово,Большая кольцевая,2022-09-12 05:00:13,216.316667,8001
1,2,СК учащегося 30 дней,Войковская ( Южный ),Войковская,Замоскворецкая,2022-09-12 05:00:54,648.183333,9011
2,3,БСК дружинника г.Москвы,Войковская ( Южный ),Войковская,Замоскворецкая,2022-09-12 05:00:55,865.333333,7022
3,4,30 дней,Войковская ( Южный ),Войковская,Замоскворецкая,2022-09-12 05:01:13,1048.233333,2022
4,5,КОШЕЛЕК,Войковская ( Южный ),Войковская,Замоскворецкая,2022-09-12 05:02:55,965.600000,2017


Далее введём в рассмотрение 2 модели: регрессии - для предсказывания времени(time_to_under) и классификации - для предсказывания станции захода(label). 

Обе модели будут созданы на основе моделей CatBoost.

Для начала рассмотрим модель регрессии

In [10]:
class CustomCatBoostRegressor(CatBoostRegressor):
    def __init__(self, iterations=1000):
        super(CatBoostRegressor, self).__init__()
        '''
        iterations: число деревьев
        '''
        self.y = None
        self.x = None
        self.iterations = iterations

    # кросс-валидация для определения оптимального числа деревьев
    def launch_cv(self, metric='MAPE'):
        cv_data = cv(params={'loss_function': metric, 'iterations': self.iterations, 'random_seed': 0, 'depth': 10},
                     pool=Pool(self.x, label=self.y,
                     cat_features=['ticket_type_nm', 'station_nm', 'line_nm', 'entrance_nm']),
                     fold_count=2, inverted=False, shuffle=True, partition_random_seed=0, stratified=False)
        self.set_params(iterations=np.argmin(cv_data['test-'+metric+'-mean']))
    
    # улучшенный fit, деревьев можно задать много, но выбираем лучшее количество - не переобучимся
    def fit(self, X, y=None, cat_features=None, text_features=None, embedding_features=None,
            sample_weight=None, baseline=None, use_best_model=None,
            eval_set=None, verbose=None, logging_level=None, plot=False, plot_file=None, column_description=None,
            verbose_eval=None, metric_period=None, silent=None, early_stopping_rounds=None,
            save_snapshot=None, snapshot_file=None, snapshot_interval=None, init_model=None, callbacks=None,
            log_cout=sys.stdout, log_cerr=sys.stderr):

        self.x = X
        self.y = y
        self.launch_cv()
        self.set_params(depth=10)
        super().fit(self.x, self.y, cat_features=['ticket_type_nm', 'station_nm', 'line_nm', 'entrance_nm'])



In [11]:
# Регрессия
# метод подготовки данных: удаление столбцов и строк с nan значениями при их наличии
def preprocess_regress(df):
    bad_columns = ['time_to_under', 'id','label','ticket_id', 'entrance_id', 'station_id','line_id']
    df = df.drop(columns=[x for x in bad_columns if x in df.columns])
    return df

# метод получения временных признаков
def get_time_features(df):
    df['day'] = df.pass_dttm.dt.dayofweek #день недели
    df['hour'] = df.pass_dttm.dt.hour # час в формате 24
    # разбиваем на промежутки активности пользования метро в течение дня, 
    df['shift'] = df['hour'].apply(lambda x: 0 if 10 <= x <= 17 else (
        1 if 0 <= x <= 6 else (2 if 7 <= x <= 9 else (3 if x >= 18 else x))))
    df['workday'] = df['day'].apply(lambda x: 0 if x == 5 or x == 6 else 1)

    df = df.drop(columns=['pass_dttm'])
    return df

df.dropna(inplace=True)
df = df.loc[df.time_to_under > 0]

x_regress_train, x_regress_test, y_regress_train, y_regress_test = train_test_split(df.drop(columns=['time_to_under']), df[['time_to_under']], test_size=0.3)

pipe_regression = Pipeline(steps=[('preprocess',FunctionTransformer(preprocess_regress)),
                                  ('time_features',FunctionTransformer(get_time_features)),
                                  ('model', CustomCatBoostRegressor(1000))])
pipe_regression.fit(x_regress_train, y_regress_train)
forecast_regress = pipe_regression.predict(x_regress_test)


Training on fold [0/2]
0:	learn: 0.9695006	test: 0.9693538	best: 0.9693538 (0)	total: 860ms	remaining: 14m 18s
1:	learn: 0.9414673	test: 0.9411753	best: 0.9411753 (1)	total: 1.78s	remaining: 14m 46s
2:	learn: 0.9304053	test: 0.9301109	best: 0.9301109 (2)	total: 2.34s	remaining: 12m 56s
3:	learn: 0.9159702	test: 0.9156819	best: 0.9156819 (3)	total: 3.03s	remaining: 12m 34s
4:	learn: 0.9047698	test: 0.9045032	best: 0.9045032 (4)	total: 3.58s	remaining: 11m 51s
5:	learn: 0.8982251	test: 0.8979783	best: 0.8979783 (5)	total: 4.17s	remaining: 11m 31s
6:	learn: 0.8919248	test: 0.8922558	best: 0.8922558 (6)	total: 5.11s	remaining: 12m 4s
7:	learn: 0.8810166	test: 0.8813572	best: 0.8813572 (7)	total: 5.76s	remaining: 11m 54s
8:	learn: 0.8709302	test: 0.8712824	best: 0.8712824 (8)	total: 6.47s	remaining: 11m 52s
9:	learn: 0.8671973	test: 0.8675645	best: 0.8675645 (9)	total: 6.92s	remaining: 11m 25s
10:	learn: 0.8639042	test: 0.8642828	best: 0.8642828 (10)	total: 7.31s	remaining: 10m 57s
11:	lear

Затем рассмотрим модель классификации

In [12]:
class CustomCatBoostClassifier(CatBoostClassifier):
    def __init__(self, iterations=1000):
        '''
        iterations: число деревьев
        '''
        super().__init__()
        self.y = None
        self.x = None
        self.iterations = iterations 
    
    # метод создающий данные для валидации
    def set_eval_data(self):
        df = self.x.join(self.y)
        df = df.drop_duplicates(subset=['label'])
        return df.drop(columns=['label']), df[['label']]
    
    # улучшенный fit, деревьев можно задать много, но выбираем лучшее количество - не переобучимся
    def fit(self, X, y=None, cat_features=None, text_features=None, embedding_features=None, sample_weight=None,
            baseline=None, use_best_model=None,
            eval_set=None, verbose=None, logging_level=None, plot=False, plot_file=None, column_description=None,
            verbose_eval=None, metric_period=None, silent=None, early_stopping_rounds=None,
            save_snapshot=None, snapshot_file=None, snapshot_interval=None, init_model=None, callbacks=None,
            log_cout=sys.stdout, log_cerr=sys.stderr):

        self.set_params(iterations=self.iterations, loss_function='MultiClass')
        self.x = X
        self.y = y
        self.set_params(depth=10)
        eval_x, eval_y = self.set_eval_data()
        super().fit(self.x, self.y,
                    cat_features=['ticket_type_nm','station_nm', 'line_nm', 'entrance_nm'],
                    eval_set=(eval_x, eval_y),
                    use_best_model=True)


In [13]:
# Классификация
# метод подготовки данных: удаление столбцов и строк с nan значениями при их наличии
def preprocess_multiclass(df):
    bad_columns = ['time_to_under', 'pass_dttm','id', 'ticket_id', 'entrance_id', 'station_id', 'line_id']

    df = df.drop(columns=[x for x in bad_columns if x in df.columns])
    return df

x_multiclass_train, x_multiclass_test, y_multiclass_train, y_multiclass_test = train_test_split(df.drop(columns=['label'])[:600], df[['label']][:600], test_size=0.3, random_state=42)

pipe_regression = Pipeline(steps=[('preprocess', FunctionTransformer(preprocess_multiclass)),
                                  ('model', CustomCatBoostClassifier(100))])
pipe_regression.fit(x_multiclass_train, y_multiclass_train)
forecast_class = pipe_regression.predict(x_multiclass_test)

Learning rate set to 0.255737
0:	learn: 5.1003544	test: 5.1678471	best: 5.1678471 (0)	total: 1m 10s	remaining: 1h 56m 56s
1:	learn: 5.0063697	test: 5.1515150	best: 5.1515150 (1)	total: 2m 17s	remaining: 1h 51m 55s
2:	learn: 4.9193104	test: 5.1117453	best: 5.1117453 (2)	total: 3m 21s	remaining: 1h 48m 21s
3:	learn: 4.8363896	test: 5.0862784	best: 5.0862784 (3)	total: 4m 26s	remaining: 1h 46m 40s
4:	learn: 4.7440869	test: 5.0617129	best: 5.0617129 (4)	total: 5m 31s	remaining: 1h 44m 58s
5:	learn: 4.6560307	test: 5.0509398	best: 5.0509398 (5)	total: 6m 35s	remaining: 1h 43m 9s
6:	learn: 4.5665198	test: 5.0342685	best: 5.0342685 (6)	total: 7m 39s	remaining: 1h 41m 44s
7:	learn: 4.4753912	test: 5.0223453	best: 5.0223453 (7)	total: 8m 44s	remaining: 1h 40m 28s
8:	learn: 4.3864531	test: 5.0006312	best: 5.0006312 (8)	total: 9m 47s	remaining: 1h 39m
9:	learn: 4.2989121	test: 4.9827915	best: 4.9827915 (9)	total: 10m 51s	remaining: 1h 37m 41s
10:	learn: 4.2119462	test: 4.9638061	best: 4.9638061 (

Рассчитаем метрики

In [14]:
from sklearn.metrics import r2_score, recall_score

def result(actual_class, forecast_class, actual_regress, forecast_regress):
    print('recall: ',recall_score(actual_class, forecast_class, average='micro'))
    print('R2: ',r2_score(actual_regress, forecast_regress))
    return 0.5 * r2_score(actual_regress, forecast_regress) + 0.5 * recall_score(actual_class, forecast_class, average='micro')


In [15]:
final = result(y_multiclass_test, forecast_class, y_regress_test, forecast_regress)
print('result: ', final)

recall:  0.06111111111111111
R2:  0.5369739279902205
result:  0.2990425195506658
